# **Data-Driven Spatio-Temporal Reef Insights**
### **Data Collection, Compilation and Evaluation from the GBR**

In [4]:
%pip install folium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.1/110.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.1/85.1 kB 26.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


This is a preliminary document that involves the loading, compilation and augmentation of multiple sources of data to combine into an "ideal set" to be used later throughout the project: There are 2 components to the data collection: the **primary data** and the **auxiliary data**. The usage of auxiliary data was inspired by the following [research article](https://www.nature.com/articles/s41598-024-75385-2) *(this is the same research article used by me in the paper extension 3, and may be referenced again)*. 

### Primary Data Sourcing and Compilation

The primary data is composed of the **Long Term Monitoring Summarised by Broad Benthic Groups** *(which can be found in this [link](https://apps.aims.gov.au/metadata/view/5a8a4b00-4ade-11dc-8f56-00008a07204e))*. Here are the details:

<div style="font-family: 'Times New Roman'; margin-left: 1.5rem">
Benthic organisms were surveyed annually on fixed sites in one habitat on each of 47 selected core survey reefs from 1993 to 2005 in 6 regions throughout the Great Barrier Reef (GBR). Surveys were undertaken at 3 sites per reef, with 5x50m transects surveyed per site.

In 2004 a new zoning plan was implemented in the Great Barrier reef Marine Park and in 2006 the pattern of surveys was changed. The original set of reefs (47) are surveyed in odd years (e.g. 2007) and a different set (56 reefs) are surveyed in even years. The new set consists of paired reefs (one no-take, and one open to fishing) surveyed to assess the effects of rezoning. The paired reefs are near Cairns and Innisfail, Townsville, Mackay, the Swain Reefs and the Capricorn-Brunker group.

From 1993 to 2006 video transects were recorded and sub-sampled at 200 points per sample. The program was modified in 2007 to allow collection of data from single frames shot at 1m intervals along each transect using a digital still camera.

Benthic groups recorded are: Abiotic; Hard coral; Soft coral; Coralline algae; Macro algae; Turf algae; Sponge; Other; Indeterminate.
</div>

The details mention that the data has **7 distinct categories**, though the data which was publicly available for download only had the following categories:

* **Hard coral**; 
* **Soft coral**; 
* **Algae**;
* **Other**.

It should be noted that I'm going to assume the category **`Algae`** is inclusive of **coralline algae**, **macro algae**, and **turf algae**.

It was also brought to my knowledge that the **`Other`** category consists of **sponges** and **other stationary organisms**. It is not clear yet on whether **`Other`** includes empty substrate space or not, however since our study is focused on the **shift** of benthic group distributions rather than the amount, this shouldn't really matter.

I'm going to start off by **loading the primary data** as a *Pandas DF* and display its contents:

In [6]:
import pandas as pd
# Disable `SettingWithCopyWarning` globally
pd.options.mode.chained_assignment = None

primary_df = pd.read_csv('./data/ltmp_summarised_by_benthic_group/data.csv')
primary_df

,SECTOR,SHELF,REEF_NAME,REEF_ID,SITE_NO,LATITUDE,LONGITUDE,VISIT_NO,YEAR_CODE,SAMPLE_DATE,GROUP_CODE,COVER
0,CA,I,LOW ISLANDS REEF,16028S,1,-16.383517,145.570950,1,199293,1993-06-12,Algae,30.64
1,CA,I,LOW ISLANDS REEF,16028S,1,-16.383517,145.570950,1,199293,1993-06-12,Hard Coral,24.77
2,CA,I,LOW ISLANDS REEF,16028S,1,-16.383517,145.570950,1,199293,1993-06-12,Other,26.78
3,CA,I,LOW ISLANDS REEF,16028S,1,-16.383517,145.570950,1,199293,1993-06-12,Soft Coral,17.82
4,CA,I,LOW ISLANDS REEF,16028S,2,-16.386483,145.572600,1,199293,1993-06-12,Algae,35.83
...,...,...,...,...,...,...,...,...,...,...,...,...
18535,WH,O,REBE REEF,19209S,2,-19.797350,150.163067,31,202223,2023-02-02,Soft Coral,20.70
18536,WH,O,REBE REEF,19209S,3,-19.800517,150.162350,31,202223,2023-02-02,Algae,51.27
18537,WH,O,REBE REEF,19209S,3,-19.800517,150.162350,31,202223,2023-02-02,Hard Coral,27.22
18538,WH,O,REBE REEF,19209S,3,-19.800517,150.162350,31,202223,2023-02-02,Other,3.79


The first thing to note here is **`SITE_NO`**; apparantly, certain reefs may have multiple sites within them. Since these sites have very little spatial information between them, and since they inherently belong to the same "reef cluster", I'm going to **reduce** the dataset and effectively get rid of **`SITE_NO`** *by assigning the mean **`COVER`** to each benthic group, ensuring that it all adds up to **100%***. 

I did have quite a hard time figuring this one out since it involved so many peculiar transformations, so I did use *ChatGPT 4o* to help me do this part:

In [4]:
import pandas as pd

# Group by `REEF_NAME`, `GROUP_CODE`, and `YEAR` to calculate the mean `COVER`;
    # (aggregate other columns by their first value while preserving order);
aggregated = (
    primary_df
    .sort_values(by=['REEF_NAME', 'GROUP_CODE', 'YEAR_CODE']) # Sort by group keys to maintain order;
    .groupby(['REEF_NAME', 'GROUP_CODE', 'YEAR_CODE'], sort=False)
    .agg({
        'COVER': 'mean', # Average the `COVER` values;
        **{col: 'first' for col in primary_df.columns if col not in ['REEF_NAME', 'GROUP_CODE', 'COVER', 'SITE_NO', 'YEAR_CODE']}
    })
    .reset_index() # Reset index to maintain a clean DataFrame;
)

# Normalise `COVER` to ensure it sums to 100% within each `REEF_NAME` and `YEAR`;
aggregated['COVER'] = (
    aggregated.groupby(['REEF_NAME', 'YEAR_CODE'])['COVER']
    .transform(lambda x: (x / x.sum()) * 100)
)

# Ensure original row order based on `REEF_NAME`, `GROUP_CODE`, and `YEAR` appearance in `primary_df`;
    # (also avoid potential duplicates in the merged DataFrame);
ordered = primary_df[['REEF_NAME', 'GROUP_CODE', 'YEAR_CODE']].drop_duplicates().reset_index(drop=True)
primary_condensed_df = (
    ordered.merge(aggregated, on=['REEF_NAME', 'GROUP_CODE', 'YEAR_CODE'], how='left', suffixes=('', '_agg'))
)

# Display;
primary_condensed_df

,REEF_NAME,GROUP_CODE,YEAR_CODE,COVER,SECTOR,SHELF,REEF_ID,LATITUDE,LONGITUDE,VISIT_NO,SAMPLE_DATE
0,LOW ISLANDS REEF,Algae,199293,34.678844,CA,I,16028S,-16.383517,145.57095,1,1993-06-12
1,LOW ISLANDS REEF,Hard Coral,199293,30.162328,CA,I,16028S,-16.383517,145.57095,1,1993-06-12
2,LOW ISLANDS REEF,Other,199293,21.122629,CA,I,16028S,-16.383517,145.57095,1,1993-06-12
3,LOW ISLANDS REEF,Soft Coral,199293,14.036199,CA,I,16028S,-16.383517,145.57095,1,1993-06-12
4,LOW ISLANDS REEF,Algae,199495,31.991201,CA,I,16028S,-16.383517,145.57095,3,1995-04-15
...,...,...,...,...,...,...,...,...,...,...,...
6179,REBE REEF,Soft Coral,202122,13.046667,WH,O,19209S,-19.791917,150.16125,30,2022-02-24
6180,REBE REEF,Algae,202223,43.643333,WH,O,19209S,-19.791917,150.16125,31,2023-02-01
6181,REBE REEF,Hard Coral,202223,35.450000,WH,O,19209S,-19.791917,150.16125,31,2023-02-01
6182,REBE REEF,Other,202223,3.796667,WH,O,19209S,-19.791917,150.16125,31,2023-02-01


I will also get rid of **`VISIT_NO`**, since **`YEAR_CODE`**, and **`SAMPLE_DATE`** are the only temporal indicators which could be used in the upcoming project:

In [5]:
primary_condensed_df = primary_condensed_df.drop(columns=['VISIT_NO'])
primary_condensed_df

,REEF_NAME,GROUP_CODE,YEAR_CODE,COVER,SECTOR,SHELF,REEF_ID,LATITUDE,LONGITUDE,SAMPLE_DATE
0,LOW ISLANDS REEF,Algae,199293,34.678844,CA,I,16028S,-16.383517,145.57095,1993-06-12
1,LOW ISLANDS REEF,Hard Coral,199293,30.162328,CA,I,16028S,-16.383517,145.57095,1993-06-12
2,LOW ISLANDS REEF,Other,199293,21.122629,CA,I,16028S,-16.383517,145.57095,1993-06-12
3,LOW ISLANDS REEF,Soft Coral,199293,14.036199,CA,I,16028S,-16.383517,145.57095,1993-06-12
4,LOW ISLANDS REEF,Algae,199495,31.991201,CA,I,16028S,-16.383517,145.57095,1995-04-15
...,...,...,...,...,...,...,...,...,...,...
6179,REBE REEF,Soft Coral,202122,13.046667,WH,O,19209S,-19.791917,150.16125,2022-02-24
6180,REBE REEF,Algae,202223,43.643333,WH,O,19209S,-19.791917,150.16125,2023-02-01
6181,REBE REEF,Hard Coral,202223,35.450000,WH,O,19209S,-19.791917,150.16125,2023-02-01
6182,REBE REEF,Other,202223,3.796667,WH,O,19209S,-19.791917,150.16125,2023-02-01


At this point, I should note that different **spatio-temporal modelling methodologies** will require slightly different data preperation techniques, and give a reminder this file is *just to collect the data into usable and coherent CSV files* so that I don't waste time ding basic augmentations in the actual **`proj.ipynb`** notebook. Thus, the only final modification I will make to the **primary data** is turning it into a viable time-series like dataset.

To do this, I'm going to further **augment** the data so that the **`GROUP_CODE`** effectively becomes **`[GROUP_CODE]_COVER`**. This is necessary as features which relate to one specific temporal and spatial point shouldn't have its crucial data split into 4 rows *(ie. `Algae`, `Hard Coral`, `Other` and `Soft Coral`)*:

In [6]:
# Pivot the DataFrame to reshape `GROUP_CODE` into individual `COVER` columns;
st_table = primary_condensed_df.pivot_table(
    index=['LATITUDE', 'LONGITUDE', 'SAMPLE_DATE'], # Group by spatial and temporal points;
    columns='GROUP_CODE', # `GROUP_CODE` becomes new columns;
    values='COVER', # Values to fill the new columns;
    aggfunc='mean' # Aggregate duplicates by mean;
).reset_index()

# Rename columns to include '[...]_COVER' suffix for `GROUP_CODE` columns;
st_table.columns = [
    f"{col.upper()}_COVER" if col not in ['LATITUDE', 'LONGITUDE', 'SAMPLE_DATE'] else col.upper()
    for col in st_table.columns
]

# Merge the reshaped DataFrame with the original data to retain other features;
primary_finalised_df = primary_condensed_df.merge(
    st_table, 
    on=['LATITUDE', 'LONGITUDE', 'SAMPLE_DATE'], 
    how='left'
)

# Drop the original 'COVER' column and remove duplicates caused by the merge and display;
primary_finalised_df = primary_finalised_df.drop(columns=['COVER', 'GROUP_CODE']).drop_duplicates()
primary_finalised_df

,REEF_NAME,YEAR_CODE,SECTOR,SHELF,REEF_ID,LATITUDE,LONGITUDE,SAMPLE_DATE,ALGAE_COVER,HARD CORAL_COVER,OTHER_COVER,SOFT CORAL_COVER
0,LOW ISLANDS REEF,199293,CA,I,16028S,-16.383517,145.57095,1993-06-12,34.678844,30.162328,21.122629,14.036199
4,LOW ISLANDS REEF,199495,CA,I,16028S,-16.383517,145.57095,1995-04-15,31.991201,34.674355,16.258916,17.075528
8,LOW ISLANDS REEF,199596,CA,I,16028S,-16.383517,145.57095,1996-01-07,35.421181,39.251308,14.013800,11.313710
12,LOW ISLANDS REEF,199697,CA,I,16028S,-16.383517,145.57095,1997-01-29,37.783333,35.430000,16.686667,10.100000
16,LOW ISLANDS REEF,199798,CA,I,16028S,-16.383517,145.57095,1998-02-01,43.055231,30.345655,13.476217,13.122896
...,...,...,...,...,...,...,...,...,...,...,...,...
6164,REBE REEF,201617,WH,O,19209S,-19.791917,150.16125,2017-02-21,81.030000,13.536667,3.233333,2.200000
6168,REBE REEF,201819,WH,O,19209S,-19.791917,150.16125,2019-04-26,69.241025,20.622646,4.773174,5.363155
6172,REBE REEF,202021,WH,O,19209S,-19.791917,150.16125,2021-02-07,51.771135,31.293945,5.361725,11.573195
6176,REBE REEF,202122,WH,O,19209S,-19.791917,150.16125,2022-02-24,45.350000,37.683333,3.920000,13.046667


This is our **full size primary dataset**, and I'll save it into a *CSV* for future reference:

In [7]:
primary_finalised_df.to_csv('./processed_data/full_size/primary_ltmp_summarised_by_benthic_group.csv', index=False)

### Truncating the Primary Data

Up till now, everything has been going well, though I noticed that there is quite a large issue at hand: there are some spatial locations which only have a few (less than 10) temporal intervals. This is problematic due to the fact that the data is yearly to begin with, so reef locations with only a few data points wouldn't allow us to derive any information out of them.

The total number of reefs are:

In [8]:
print(len(primary_finalised_df['REEF_NAME'].unique()))

103


I'll first compile an initial dataframe:

In [9]:
# Get unique latitude and longitude combinations;
unique_locations = primary_finalised_df[['REEF_NAME', 'SECTOR', 'LATITUDE', 'LONGITUDE']].drop_duplicates()
unique_locations.columns = ['name', 'sector', 'lat', 'lon']
# Display;
unique_locations

,name,sector,lat,lon
0,LOW ISLANDS REEF,CA,-16.383517,145.570950
88,GREEN ISLAND REEF,CA,-16.775317,145.980450
172,FITZROY ISLAND REEF,CA,-16.923150,145.993133
260,MACKAY REEF,CA,-16.042633,145.654550
344,HASTINGS REEF,CA,-16.499350,146.022667
...,...,...,...,...
5744,19138S,WH,-19.806867,149.430467
5836,20104S,WH,-20.029433,149.695100
5916,SLATE REEF,WH,-19.662850,149.915167
6004,HYDE REEF,WH,-19.738717,150.080800


And I'll limit the spatial locations to only the reefs which have **20 temporal intervals** or more:

In [33]:
# Filter names with more than 20 unique `SAMPLE_DATE`s;
selected_locations = (
    primary_finalised_df.groupby('REEF_NAME')['SAMPLE_DATE']
    .nunique() # Count unique `SAMPLE_DATE`s per `REEF_NAME`;
    .reset_index(name='num_intervals') # Rename the count column;
    .query('num_intervals >= 20') # Keep only reefs with >= 20 temporal intervals;
)

# Filter `unique_locations` according to the `selected_locations`;
selected_locations.columns = ['name', 'n']
selected_locations = unique_locations.merge(
    selected_locations[['name']],
    on='name',
    how='inner'
).reset_index(drop=True)

selected_locations

,name,sector,lat,lon
0,LOW ISLANDS REEF,CA,-16.383517,145.570950
1,GREEN ISLAND REEF,CA,-16.775317,145.980450
2,FITZROY ISLAND REEF,CA,-16.923150,145.993133
3,MACKAY REEF,CA,-16.042633,145.654550
4,HASTINGS REEF,CA,-16.499350,146.022667
5,MICHAELMAS REEF,CA,-16.548650,146.050000
6,THETFORD REEF,CA,-16.798800,146.198117
7,AGINCOURT REEFS (NO 1),CA,-16.038383,145.868767
8,ST CRISPIN REEF,CA,-16.071583,145.845150
9,BROOMFIELD REEF,CB,-23.249567,151.942550


Finally, based on these **selected locations** I will now **truncate** the dataset:

In [25]:
primary_truncated_df = primary_finalised_df[primary_finalised_df['REEF_NAME'].isin(selected_locations['name'])]
primary_truncated_df

,REEF_NAME,YEAR_CODE,SECTOR,SHELF,REEF_ID,LATITUDE,LONGITUDE,SAMPLE_DATE,ALGAE_COVER,HARD CORAL_COVER,OTHER_COVER,SOFT CORAL_COVER
0,LOW ISLANDS REEF,199293,CA,I,16028S,-16.383517,145.57095,1993-06-12,34.678844,30.162328,21.122629,14.036199
4,LOW ISLANDS REEF,199495,CA,I,16028S,-16.383517,145.57095,1995-04-15,31.991201,34.674355,16.258916,17.075528
8,LOW ISLANDS REEF,199596,CA,I,16028S,-16.383517,145.57095,1996-01-07,35.421181,39.251308,14.013800,11.313710
12,LOW ISLANDS REEF,199697,CA,I,16028S,-16.383517,145.57095,1997-01-29,37.783333,35.430000,16.686667,10.100000
16,LOW ISLANDS REEF,199798,CA,I,16028S,-16.383517,145.57095,1998-02-01,43.055231,30.345655,13.476217,13.122896
...,...,...,...,...,...,...,...,...,...,...,...,...
6164,REBE REEF,201617,WH,O,19209S,-19.791917,150.16125,2017-02-21,81.030000,13.536667,3.233333,2.200000
6168,REBE REEF,201819,WH,O,19209S,-19.791917,150.16125,2019-04-26,69.241025,20.622646,4.773174,5.363155
6172,REBE REEF,202021,WH,O,19209S,-19.791917,150.16125,2021-02-07,51.771135,31.293945,5.361725,11.573195
6176,REBE REEF,202122,WH,O,19209S,-19.791917,150.16125,2022-02-24,45.350000,37.683333,3.920000,13.046667


And finish it all off by saving it into a *CSV* file:

In [26]:
primary_truncated_df.to_csv('./processed_data/truncated/primary_ltmp_summarised_by_benthic_group.csv')

I may also need to use the **`selected_locations`** for the sake of retrieving the **auxiliary data** in later parts, so I'll save that aswell:

In [27]:
selected_locations.to_csv('./processed_data/truncated/centralised_locations.csv')

### Auxiliary Data Sourcing (Hydro and Rivertracing)

The auxiliary data is composed of **grid-based spatio-temporal data** from the **eReefs** platform; most of this grid data is available for public downlaod in **4km grid** format.

The issue is that, *since the data is grid-based*, we can approach it by either:

* Getting temporal insights based on the positions of the **reefs** from the LTMP data.
* Getting **random points** across the general surrounding area and implement that as auxiliary data.

Since we have the choice, I'll do both: auxiliary data **centralised** around reefs, and auxiliary data **de-centralised** around reefs. Please note that I have already saved the **centralised locations** in the `./processed_data/truncated/centralised_locations.csv` *CSV* file.

This should also provide an interesting perspective as we would be able to cover whether or not introducing auxiliary data at spatial points *between* primary data points would improve the model or not *(of course such observations would be more related to the data itself and wouldn't exactly be a "universal" observation, but it would be interesting regardless)*.

Since we already have the **centralised auxiliary data locations**, all that is left is for the **de-centralised locations**. I plan on manually selecting these through an interactive **`folium`** map, which will allow me to *select and retrieve the **spatial details** of those selections*. 

However, *since **`folium`** doesn't have a two-way Python API, I will have to "inject" some HTML into the interactive webview, download that webview and retrieve the locations through the `console.log(...)` iterations*. Here is my implementation of this solution:

In [28]:
import folium

# Initialise a base map;
m = folium.Map(location=[-16.9, 145.7], zoom_start=6)
# Add existing points from `selected_locations` to the map as purple points;
for _, row in selected_locations.iterrows():
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=row['name'],
        icon=folium.Icon(color='purple')
    ).add_to(m)

# Enable "click to add marker" functionality (in JS);
click_js = """
<script>
    document.addEventListener('DOMContentLoaded', function() {
        // Locate the Folium Leaflet map instance
        var leafletMap = Object.values(window).find(o => o instanceof L.Map);

        leafletMap.on('click', function(e) {
            // Add a default marker at the clicked location
            var marker = L.marker(e.latlng).addTo(leafletMap);
            console.log("Lat: " + e.latlng.lat + ", Lng: " + e.latlng.lng);
        });
    });
</script>
"""
# Attach the custom JavaScript to the map;
m.get_root().html.add_child(folium.Element(click_js))

# Save as HTML;
m.save('folium_click_map.html')

After selecting some points on the map, I gave the console logs to *GPT 4o* to transform them into the same CSV-like format we used for **`selected_positions`** *(that is, `name`, `lat`, and `lon`)*. I copy-pasted this output into the file `./processed_data/truncated/decentralised_locations.csv`. 

The file `./processed_data/decentralised_locations.csv` now effectively becomes our **de-centralised auxiliary data points**.

I'll load this now, and show their corresponding points on the map within this Jupyter notebook:

In [29]:
import folium

# Load the external points;
decentralised_locations = pd.read_csv('./processed_data/truncated/decentralised_locations.csv')

# Initialise a base map;
m = folium.Map(location=[-16.9, 145.7], zoom_start=6)
# Add existing points from `unique_locations` to the map in blue;
for _, row in selected_locations.iterrows():
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=row['name'],
        icon=folium.Icon(color='purple')
    ).add_to(m)
# Add the new external points onto the map;
for _, row in decentralised_locations.iterrows():
    folium.Marker(location=[row['lat'], row['lon']],popup=row['name']).add_to(m)

# Display map on notebook;
m

Now, at this stage all that is left is to merge the **centralised_locations** *(ie. `selected_locations`)* and **`decentralised_locations`** into a final **`auxiliary_locations`** DF; this object will be given to the **eReefs platform** for the data extraction:

In [32]:
centralised_locations = selected_locations.drop(columns=['sector'])
auxiliary_locations = pd.concat([centralised_locations, decentralised_locations], ignore_index=True)
auxiliary_locations.to_csv('processed_data/truncated/auxiliary_locations.csv')

auxiliary_locations

,name,lat,lon
0,LOW ISLANDS REEF,-16.383517,145.570950
1,GREEN ISLAND REEF,-16.775317,145.980450
2,FITZROY ISLAND REEF,-16.923150,145.993133
3,MACKAY REEF,-16.042633,145.654550
4,HASTINGS REEF,-16.499350,146.022667
...,...,...,...
59,SpP_16,-23.155513,152.693481
60,SpP_17,-20.509355,151.798096
61,SpP_18,-17.790535,147.293701
62,SpP_19,-24.106647,152.720947


The link to the data extraction tool can be found [here](https://extraction.ereefs.aims.gov.au/extract-data).

It does take some time for the data to extract *(according to the **eReefs platform**, this could be for a few hours)*. Once the extraction is completed I will handle the compilation of the data in the following section.

### Auxiliary Data Compilation (Hydro and Rivertracing)

The data has been successfully extracted from **eReefs**, and now ready to be compiled. However, before any compilation takes place there are some details which need to be clarified:

First of all, it should be noted that there are **3 types of data** that are provided by the **eReefs** platform:

* **GBR4 Bigeochemistry and ecology**, which is focused on chemical characteristics such as Ammonia, Nitrogen, etc; on a **4km grid** basis.
* **GBR4 Hydrodynamics** which covers aspects such as current, temperature and salinity; on a **4km grid** basis.
* **GBR4 Rivertracing** which tracks how riverine inputs disperse across the oceans *(this is a more peculiar type of data since it is a lot more reliant on calculations rather than actual sensors, but it should still be taken into account since potential pollutants from certain rivers could be cause for benthic distribution shifts)*; on a **4km grid** basis.

All 3 are viable data sources in theory, but there is a warning that **eReefs** has issued specifically for the **bigeochemistry and ecology** data:

<div style="font-family: 'Times New Roman'; margin-left: 1.5rem">
Please be aware that this version (3.1) of the model was setup for scenario comparisons. River loads are based off 2019 land practices over the complete history (2010 - 2019). Using this version to estimate historic water quality parameters needs to be done with caution.

<span style="margin-left: 1.5rem">*Source: [eReefs FAQ](https://extraction.ereefs.aims.gov.au/faqs#faq6)*</span>
</div>

Essentially the data is to be considered *"low-quality"* for any temporal purpose, or purpose in which it has to be taken into account in a historical sense. While doing my research, I decided that I didn't want to introduce **auxiliary data** that would introduce inaccurate metrics, so I decided against using **bigeochemistry and ecology** altogether.

It should also be noted that the data extracted starts at the **1st of January 2011** and ends at the **31st of December 2022**. Some of the **LTMP primary data** is outside of these ranges. The data has also been aquired on a **daily** basis, which I will resample to a **weekly** one.

First, lets load the **GBR4 Rivertracing** data:

In [7]:
gbr4_rivertracing = pd.read_csv('./data/ereefs_auxiliary_rivertracing/ereefs_gbr4_rivertracing.csv', low_memory=False)
gbr4_rivertracing

,Aggregated Date/Time,Variable,Depth,Site Name,Latitude,Longitude,mean,median,p5,p95,lowest,highest
0,2010-12-01T00:00,passive,-1.5,SpP_1,-14.108608,143.940125,0.0,0.0,0.0,0.0,0.0,0.0
1,2010-12-01T00:00,passive,-1.5,SpP_2,-13.239945,143.915405,0.0,0.0,0.0,0.0,0.0,0.0
2,2010-12-01T00:00,passive,-1.5,SpP_3,-11.652236,143.871460,0.0,0.0,0.0,0.0,0.0,0.0
3,2010-12-01T00:00,passive,-1.5,SpP_4,-10.698394,143.657227,0.0,0.0,0.0,0.0,0.0,0.0
4,2010-12-01T00:00,passive,-1.5,SpP_5,-14.445319,145.140381,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3680251,2024-01-17T00:00,pin,-1.5,19138S,-19.806867,149.430467,4.4052210275776216E-8,4.4052210275776216E-8,4.4052210275776216E-8,4.4052210275776216E-8,4.4052210275776216E-8,4.4052210275776216E-8
3680252,2024-01-17T00:00,pin,-1.5,20104S,-20.029433,149.695100,4.4029113504527416E-8,4.4029113504527416E-8,4.4029113504527416E-8,4.4029113504527416E-8,4.4029113504527416E-8,4.4029113504527416E-8
3680253,2024-01-17T00:00,pin,-1.5,SLATE REEF,-19.662850,149.915167,4.4229194421527555E-8,4.4229194421527555E-8,4.4229194421527555E-8,4.4229194421527555E-8,4.4229194421527555E-8,4.4229194421527555E-8
3680254,2024-01-17T00:00,pin,-1.5,HYDE REEF,-19.738717,150.080800,4.427449463104556E-8,4.427449463104556E-8,4.427449463104556E-8,4.427449463104556E-8,4.427449463104556E-8,4.427449463104556E-8


And then the **GBR4 Hydrodynamics** data:

In [8]:
gbr4_hydro = pd.read_csv('./data/ereefs_auxiliary_hydro/ereefs_gbr4_hydro.csv', low_memory=False)
gbr4_hydro

,Aggregated Date/Time,Variable,Depth,Site Name,Latitude,Longitude,mean,median,p5,p95,lowest,highest
0,2010-09-01T00:00,salt,-1.5,SpP_1,-14.108608,143.940125,35.0693558206111,35.06845300444222,35.05829896408678,35.0799722245782,35.05810831466729,35.0799722245782
1,2010-09-01T00:00,salt,-1.5,SpP_2,-13.239945,143.915405,35.10310061011293,35.10502378080408,35.09673471634785,35.10699020826116,35.09526791347743,35.10699020826116
2,2010-09-01T00:00,salt,-1.5,SpP_3,-11.652236,143.871460,34.913862540146134,34.91586962163154,34.90455219655267,34.922804764557696,34.903173539323745,34.922804764557696
3,2010-09-01T00:00,salt,-1.5,SpP_4,-10.698394,143.657227,34.892682217488094,34.89313752751636,34.890266982734076,34.89407591762416,34.889746949246835,34.89407591762416
4,2010-09-01T00:00,salt,-1.5,SpP_5,-14.445319,145.140381,35.183705206540544,35.18377965932447,35.18068121603769,35.187254666082985,35.179596041895984,35.187254666082985
...,...,...,...,...,...,...,...,...,...,...,...,...
1876603,2024-01-17T00:00,v,-1.5,19138S,-19.806867,149.430467,-0.0074239473314560945,0.03358435140938712,-0.596827123914447,0.5521110230373204,-0.604640955907677,0.5521110230373204
1876604,2024-01-17T00:00,v,-1.5,20104S,-20.029433,149.695100,0.0061101838273970865,0.08584920694223824,-0.5377428371996893,0.5066069959648293,-0.5489613797904147,0.5066069959648293
1876605,2024-01-17T00:00,v,-1.5,SLATE REEF,-19.662850,149.915167,-0.058469700259933845,-0.0855396663014035,-0.7427242263374023,0.6722797821597638,-0.7772017084275348,0.6722797821597638
1876606,2024-01-17T00:00,v,-1.5,HYDE REEF,-19.738717,150.080800,0.00937475229408268,0.0444915635702482,-0.647616845446351,0.7410047402087071,-0.6716628970103125,0.7410047402087071


**NOTE: I did have to set `low_memory` to `False` due to how large these files were.**

First of all, I believe that it is necessary to get rid of **`lowest`** and **`highest`** from both files, especially since using **`p5`** or **`p95`** would lead to far more stable readings. I also dont really have much need for **`median`**, so I'll be getting rid of that aswell:

In [9]:
gbr4_rivertracing = gbr4_rivertracing.drop(columns=['lowest', 'highest', 'median'])
gbr4_hydro = gbr4_hydro.drop(columns=['lowest', 'highest', 'median'])

It may also be necessary to get rid of **`Depth`**, since all metrics are from surface-level depth areas *(I chose -3m so that it isn't purely surface level metrics, and can more accurately map factors such as current)*:

In [10]:
gbr4_rivertracing = gbr4_rivertracing.drop(columns=['Depth'])
gbr4_hydro = gbr4_hydro.drop(columns=['Depth'])

While analysing the data I also took note of another issue; the **`eta`** variable in the **GBR4 Hydrodynamics** data doesn't seem to have much meaning or context related to it *(at least from what I could find)*, it seems to me like some sort of internal flag used by **eReefs** and I have decided to get rid of it accordingly:

In [11]:
gbr4_hydro_adj = gbr4_hydro[gbr4_hydro['Variable'] != 'eta']
gbr4_hydro_adj

,Aggregated Date/Time,Variable,Site Name,Latitude,Longitude,mean,p5,p95
0,2010-09-01T00:00,salt,SpP_1,-14.108608,143.940125,35.0693558206111,35.05829896408678,35.0799722245782
1,2010-09-01T00:00,salt,SpP_2,-13.239945,143.915405,35.10310061011293,35.09673471634785,35.10699020826116
2,2010-09-01T00:00,salt,SpP_3,-11.652236,143.871460,34.913862540146134,34.90455219655267,34.922804764557696
3,2010-09-01T00:00,salt,SpP_4,-10.698394,143.657227,34.892682217488094,34.890266982734076,34.89407591762416
4,2010-09-01T00:00,salt,SpP_5,-14.445319,145.140381,35.183705206540544,35.18068121603769,35.187254666082985
...,...,...,...,...,...,...,...,...
1876603,2024-01-17T00:00,v,19138S,-19.806867,149.430467,-0.0074239473314560945,-0.596827123914447,0.5521110230373204
1876604,2024-01-17T00:00,v,20104S,-20.029433,149.695100,0.0061101838273970865,-0.5377428371996893,0.5066069959648293
1876605,2024-01-17T00:00,v,SLATE REEF,-19.662850,149.915167,-0.058469700259933845,-0.7427242263374023,0.6722797821597638
1876606,2024-01-17T00:00,v,HYDE REEF,-19.738717,150.080800,0.00937475229408268,-0.647616845446351,0.7410047402087071


I'm now going to **augment** the data in a similar fashion to what was done for the **primary data**; that is, I'll **pivot** the DF so that **`[Variable]`** becomes **`[Variable]_mean`**, **`[Variable]_p5`** and **`[Variable]_p95`** for *instances which are based on identical temporal/spatial points*.

I'll define a **function** to do this:

In [12]:
# IMPORTANT: Each `Site Name` has unique spatial properties, so using it as an alternative to grouping by `Longitude` and `Latitude` is okay in this context;

def pivot_auxiliary(input_df):
    # It may be necessary to explicitly define `mean`, `p5` and `p95` as numeric;
        # (pandas has a tendency to assign the `object` dtype on large datasets for the sake of memory optimisations);
    input_df['mean'] = pd.to_numeric(input_df['mean'], errors='coerce')
    input_df['p5'] = pd.to_numeric(input_df['p5'], errors='coerce')
    input_df['p95'] = pd.to_numeric(input_df['p95'], errors='coerce')

    # Pivot the DataFrame to reshape `Variable` into individual columns with suffixes for `mean`, `p5`, and `p95`;
    st_table = input_df.pivot_table(
        index=['Site Name', 'Aggregated Date/Time'], # Group by spatial and temporal points;
        columns='Variable',  # `VARIABLE` becomes new columns;
        values=['mean', 'p5', 'p95'], # Values to fill the new columns;
        aggfunc='mean' # Aggregate duplicates by mean (if necessary);
    ).reset_index()

    # Flatten the relevant columns and include suffixes;
    st_table.columns = [
        f"{var}_{stat}" if isinstance(var, str) else var
        for stat, var in st_table.columns
    ]
    # Rename misaligned columns caused by pivoting;
    st_table.rename(
        columns={'_Site Name': 'Site Name', '_Aggregated Date/Time': 'Aggregated Date/Time'},
        inplace=True
    )

    # Merge the reshaped DF with the original data to retain other features;
        # (also get rid of redundant features and duplicate rows);
    return input_df.merge(
        st_table,
        on=['Site Name', 'Aggregated Date/Time'],  # Merge on spatial and temporal points
        how='left'
    ).drop(columns=['mean', 'p5', 'p95', 'Variable']).drop_duplicates()

Then I'll apply the newly defined function onto the **GBR4 Rivertracing** dataset:

In [13]:
gbr4_rivertracing_finalised = pivot_auxiliary(gbr4_rivertracing)
gbr4_rivertracing_finalised

,Aggregated Date/Time,Site Name,Latitude,Longitude,boy_mean,dai_mean,don_mean,fit_mean,fly_mean,her_mean,...,don_p95,fit_p95,fly_p95,her_p95,log_p95,mul_p95,passive_p95,pin_p95,pio_p95,tul_p95
0,2010-12-01T00:00,SpP_1,-14.108608,143.940125,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000
1,2010-12-01T00:00,SpP_2,-13.239945,143.915405,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000
2,2010-12-01T00:00,SpP_3,-11.652236,143.871460,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000
3,2010-12-01T00:00,SpP_4,-10.698394,143.657227,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000
4,2010-12-01T00:00,SpP_5,-14.445319,145.140381,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3679547,2024-01-17T00:00,19138S,-19.806867,149.430467,7.495172e-07,0.000039,3.073328e-07,0.000027,0.000063,0.000398,...,3.073328e-07,0.000027,0.000063,0.000398,6.655474e-07,0.000467,0.001858,4.405221e-08,0.000002,0.000206
3679548,2024-01-17T00:00,20104S,-20.029433,149.695100,7.495165e-07,0.000009,3.381432e-07,0.000027,0.000064,0.000475,...,3.381432e-07,0.000027,0.000064,0.000475,6.655074e-07,0.000176,0.001173,4.402911e-08,0.000002,0.000130
3679549,2024-01-17T00:00,SLATE REEF,-19.662850,149.915167,7.495197e-07,0.000107,2.053135e-07,0.000027,0.000060,0.000011,...,2.053135e-07,0.000027,0.000060,0.000011,6.658576e-07,0.000072,0.000503,4.422919e-08,0.000002,0.000015
3679550,2024-01-17T00:00,HYDE REEF,-19.738717,150.080800,7.495207e-07,0.000101,2.055555e-07,0.000027,0.000058,0.000008,...,2.055555e-07,0.000027,0.000058,0.000008,6.659385e-07,0.000068,0.000483,4.427449e-08,0.000002,0.000015


And finally onto the **GBR4 Hydrodynamics** dataset:

In [14]:
gbr4_hydro_finalised = pivot_auxiliary(gbr4_hydro_adj)
gbr4_hydro_finalised

,Aggregated Date/Time,Site Name,Latitude,Longitude,salt_mean,temp_mean,u_mean,v_mean,wspeed_u_mean,wspeed_v_mean,...,u_p5,v_p5,wspeed_u_p5,wspeed_v_p5,salt_p95,temp_p95,u_p95,v_p95,wspeed_u_p95,wspeed_v_p95
0,2010-09-01T00:00,SpP_1,-14.108608,143.940125,35.069356,25.055796,-0.072303,-0.004626,-10.332624,3.186448,...,-0.234860,-0.207649,-12.569753,0.311640,35.079972,25.348001,0.123562,0.226765,-8.305935,5.394821
1,2010-09-01T00:00,SpP_2,-13.239945,143.915405,35.103101,25.944231,-0.097094,0.093749,-10.710273,6.540869,...,-0.238945,0.030125,-11.054252,5.784928,35.106990,26.004354,0.134960,0.238904,-9.966008,7.188743
2,2010-09-01T00:00,SpP_3,-11.652236,143.871460,34.913863,26.516123,-0.129376,0.076156,-11.161411,7.290897,...,-0.375555,-0.023787,-11.657337,6.616556,34.922805,26.628723,0.239457,0.147019,-10.357190,8.067644
3,2010-09-01T00:00,SpP_4,-10.698394,143.657227,34.892682,26.576357,-0.175196,0.064010,-11.055808,7.352576,...,-0.364740,0.000000,-11.753264,6.607400,34.894076,26.620514,0.119816,0.195366,-9.987385,8.623995
4,2010-09-01T00:00,SpP_5,-14.445319,145.140381,35.183705,25.408580,-0.190893,0.179632,-10.761263,6.398238,...,-0.358796,-0.039164,-11.623187,5.496452,35.187255,25.556245,0.089979,0.331880,-10.164009,7.512294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1876283,2024-01-17T00:00,19138S,-19.806867,149.430467,35.063816,28.342832,0.032188,-0.007424,-7.642722,2.358573,...,-0.227692,-0.596827,-8.660224,0.930005,35.081023,28.478533,0.208130,0.552111,-6.541411,3.560224
1876284,2024-01-17T00:00,20104S,-20.029433,149.695100,35.191950,28.585250,0.012589,0.006110,-7.582427,2.229005,...,-0.336805,-0.537743,-8.771539,0.867488,35.202307,28.834232,0.267330,0.506607,-6.073992,3.510290
1876285,2024-01-17T00:00,SLATE REEF,-19.662850,149.915167,35.090033,27.937987,-0.027287,-0.058470,-7.847107,1.985541,...,-0.420679,-0.742724,-8.899756,1.121231,35.115757,28.165535,0.306255,0.672280,-6.803847,3.515160
1876286,2024-01-17T00:00,HYDE REEF,-19.738717,150.080800,35.131730,27.721376,0.011142,0.009375,-7.885546,1.858646,...,-0.375952,-0.647617,-8.987440,0.992981,35.148859,27.937098,0.381558,0.741005,-6.924206,3.340505


To match with the sample dates of the primary data, I'll adjust both **`gbr4_hydro_finalised`** and **`gbr4_rivertracing_finalised`** to have a date range up until the **31st of December 2022**, starting from the **1st of December 2010**. This is also why there is a slight mismatch of rows even though the time frequency is identical; we can quickly confirm that the newly compile datasets have the same length through this line:

In [26]:
# Ensure the date column is in datetime format and truncate both the hydro and rivertracing datasets;
gbr4_hydro_finalised['Aggregated Date/Time'] = pd.to_datetime(gbr4_hydro_finalised['Aggregated Date/Time'])
gbr4_hydro_finalised = gbr4_hydro_finalised[gbr4_hydro_finalised['Aggregated Date/Time'] <= '2022-12-31']
gbr4_hydro_finalised = gbr4_hydro_finalised[gbr4_hydro_finalised['Aggregated Date/Time'] >= '2010-12-01']
gbr4_rivertracing_finalised['Aggregated Date/Time'] = pd.to_datetime(gbr4_rivertracing_finalised['Aggregated Date/Time'])
gbr4_rivertracing_finalised = gbr4_rivertracing_finalised[gbr4_rivertracing_finalised['Aggregated Date/Time'] <= '2022-12-31']
gbr4_rivertracing_finalised = gbr4_rivertracing_finalised[gbr4_rivertracing_finalised['Aggregated Date/Time'] >= '2010-12-01']

# Filter and display the relevant results;
print(f"Number of rows for the truncated `gbr4_hydro_finalised`: {gbr4_hydro_finalised.shape[0]}")
print(f"Number of rows for the truncated `gbr4_rivertracing_finalised`: {gbr4_rivertracing_finalised.shape[0]}")

Number of rows for the truncated `gbr4_hydro_finalised`: 282496
Number of rows for the truncated `gbr4_rivertracing_finalised`: 282240


There seems to be some sort of data mismatch in the number of rows; note that this is not expected since, after the datasets were pivoted, the 2 datasets should have the same date ranges, frequencies and spatial locations.

It may help to quickly **diagnose** and determine if this could be a future issue:

In [29]:
# Number of unique `Aggregated Date/Time` between the 2 dataframes (truncated for hydro is used);
print(f"# of unique temporal intervals for `gbr4_hydro_finalised`: {len(gbr4_hydro_finalised['Aggregated Date/Time'].unique())}")
print(f"# of unique temporal intervals for `gbr4_rivertracing_finalised`: {len(gbr4_rivertracing_finalised['Aggregated Date/Time'].unique())}")
# Number of unique `Site Locations` between the 2 dataframes (truncated for hydro is used);
print(f"# of unique spatial locations for `gbr4_hydro_finalised`: {len(gbr4_hydro_finalised['Site Name'].unique())}")
print(f"# of unique spatial locations for `gbr4_rivertracing_finalised`: {len(gbr4_rivertracing_finalised['Site Name'].unique())}")

# of unique temporal intervals for `gbr4_hydro_finalised`: 4414
# of unique temporal intervals for `gbr4_rivertracing_finalised`: 4410
# of unique spatial locations for `gbr4_hydro_finalised`: 64
# of unique spatial locations for `gbr4_rivertracing_finalised`: 64


It appears that the issue has to do with the **number of temporal intervals** rather than the spatial locations; However, there is one thing which we can determine pretty much immediately: If the interval starts at the **1st of January 2011**, and if it ends at the **31st of December 2022**, according to *GPT 4o* the number of days should be **4383 (inclusive)**. Since this is the case, it is obvious that **`rivertracing`** has missing data. I'm now going to determine what these missing dates are:

In [28]:
# Ensure the date columns are both in datetime format;
gbr4_hydro_finalised['Aggregated Date/Time'] = pd.to_datetime(gbr4_hydro_finalised['Aggregated Date/Time'])
gbr4_rivertracing_finalised['Aggregated Date/Time'] = pd.to_datetime(gbr4_rivertracing_finalised['Aggregated Date/Time'])
# The extract the unique dates into sets;
dateset_hydro = set(gbr4_hydro_finalised['Aggregated Date/Time'].unique())
dateset_rivertracing = set(gbr4_rivertracing_finalised['Aggregated Date/Time'].unique())

# Get the unique values in `dateset_hydro` but not in `dateset_rivertracing`;
missing_dates = dateset_hydro - dateset_rivertracing
print("The missing dates are:", missing_dates)

The missing dates are: {Timestamp('2018-12-24 00:00:00'), Timestamp('2020-01-17 00:00:00'), Timestamp('2017-05-08 00:00:00'), Timestamp('2017-11-11 00:00:00')}


It seems that the following dates are missing in the **GBR4 rivertracing** dataset:

* **8th of May 2017**,
*  **11th of November 2017**,
* **24th of December 2018**,
* and **17th of January 2020**.

However, these dates most likely represent days of IoT or equipment maintenance so I'm not that concerned.

Now, it is time for the final step: **resampling**. I'll provision **weekly resampling** and **monthly resampling** of both datasets using *Panda's built-in resampling functionality*. Note that the grouping/aggregation will happen *according to the distinct spatial/temporal properties* of the observations. I will then save it into *CSV* files. First I'll define a function for doing so:

In [30]:
def resample_auxiliary(df, freq, output_filenamedir):
    """
    Resample spatio-temporal data by a specified frequency.
    
    Parameters:
    - df: DataFrame with columns ['Site Name', 'Aggregated Date/Time', ...]
    - freq: Resampling frequency (e.g., 'W' for weekly, 'M' for monthly)
    - output_filename: Name of the output CSV file.
    """
    # Ensure `Aggregated Date/Time` is a datetime object;
    df['Aggregated Date/Time'] = pd.to_datetime(df['Aggregated Date/Time'])
    # Identify columns with `_mean`, `_p5`, or `_p95` and ensure they are numeric;
    numeric_columns = [col for col in df.columns if any(keyword in col for keyword in ['_mean', '_p5', '_p95'])]
    for col in numeric_columns:
        df[col] = pd.to_numeric(df[col], errors='coerce') # Convert to numeric, coerce invalid entries to NaN;

    # Set up the resampling with both spatial and temporal grouping;
    resampled_df = (
        df
        .set_index('Aggregated Date/Time') # Set the temporal index;
        .groupby('Site Name', group_keys=False) # Group by `Site Name`;
        .resample(freq) # Resample based on temporal frequency;
        .mean() # Aggregate by mean for all numeric columns
        .reset_index() # Reset index after resampling;
    )
    resampled_df['Site Name'] = df['Site Name']
    
    print(f"Final shape of {output_filenamedir} aggregated by {freq}: {resampled_df.shape}")
    # Save the resampled DataFrame to a CSV file;
    resampled_df.to_csv(output_filenamedir, index=False)

Then apply that function onto the **GBR4 rivertracing** dataset *(for saving both **weekly** and **monthly** resampled files)*:

In [31]:
gbr4_rivertracing_finalised.to_csv('./processed_data/full_size/ereefs_gbr4_rivertracing_d.csv')
resample_auxiliary(gbr4_rivertracing_finalised, 'W', './processed_data/truncated/ereefs_gbr4_rivertracing_w.csv')
resample_auxiliary(gbr4_rivertracing_finalised, 'ME', './processed_data/truncated/ereefs_gbr4_rivertracing_m.csv')

Final shape of ./processed_data/truncated/ereefs_gbr4_rivertracing_w.csv aggregated by W: (40384, 40)
Final shape of ./processed_data/truncated/ereefs_gbr4_rivertracing_m.csv aggregated by ME: (9280, 40)


And finally onto the **GBR4 hydrodynamics** dataset *(for saving both **weekly** and **monthly** resampled files)*:

In [32]:
gbr4_hydro_finalised.to_csv('./processed_data/full_size/ereefs_gbr4_hydro_d.csv')
resample_auxiliary(gbr4_hydro_finalised, 'W', './processed_data/truncated/ereefs_gbr4_hydro_w.csv')
resample_auxiliary(gbr4_hydro_finalised, 'ME', './processed_data/truncated/ereefs_gbr4_hydro_m.csv')

Final shape of ./processed_data/truncated/ereefs_gbr4_hydro_w.csv aggregated by W: (40384, 22)
Final shape of ./processed_data/truncated/ereefs_gbr4_hydro_m.csv aggregated by ME: (9280, 22)


## <span style="color: blue">Appendix</span>

### <span style="color: darkblue">Nuisance With Aggregation (Primary Data)</span>

For some reason, when the following **aggregation** is done by **`SAMPLE_CODE`**, it gives weird and convoluted results, where certain reefs do not match in their spatial qualities, even though **`SITE_NO`** has already been removed:

In [ ]:
import pandas as pd

# Group by `REEF_NAME`, `GROUP_CODE`, and `SAMPLE_DATE` to calculate the mean `COVER`;
    # (aggregate other columns by their first value while preserving order);
aggregated = (
    primary_df
    .sort_values(by=['REEF_NAME', 'GROUP_CODE', 'SAMPLE_DATE']) # Sort by group keys to maintain order;
    .groupby(['REEF_NAME', 'GROUP_CODE', 'SAMPLE_DATE'], sort=False)
    .agg({
        'COVER': 'mean', # Average the `COVER` values;
        **{col: 'first' for col in primary_df.columns if col not in ['REEF_NAME', 'GROUP_CODE', 'COVER', 'SITE_NO', 'SAMPLE_DATE']}
    })
    .reset_index() # Reset index to maintain a clean DataFrame;
)

# Normalise `COVER` to ensure it sums to 100% within each `REEF_NAME` and `SAMPLE_DATE`;
aggregated['COVER'] = (
    aggregated.groupby(['REEF_NAME', 'SAMPLE_DATE'])['COVER']
    .transform(lambda x: (x / x.sum()) * 100)
)

# Ensure original row order based on `REEF_NAME`, `GROUP_CODE`, and `SAMPLE_DATE` appearance in `primary_df`;
    # (also avoid potential duplicates in the merged DataFrame);
ordered = primary_df[['REEF_NAME', 'GROUP_CODE', 'SAMPLE_DATE']].drop_duplicates().reset_index(drop=True)
primary_condensed_df = (
    ordered.merge(aggregated, on=['REEF_NAME', 'GROUP_CODE', 'SAMPLE_DATE'], how='left', suffixes=('', '_agg'))
)

# Display;
primary_condensed_df

This probably has to do with some peculiarities in the data, but it may help to revisit this project and find the cause to this nuisance in the future.

### <span style="color: darkblue">Data Extraction Settings (Auxiliary; Rivertracing)</span>

When extracting the **GBR4 hydrodynamics** dataset, I selected all the available options for extraction.

However, for the **rivertracing** the amount of available options were a bit much and I decided to select around **half**. I also didn't include **temperature** or **salinity** since that was already included in the **GBR4 hydrodynamics** dataset. Here is an image of my selections:

<img src="./media/River_Tracing_GBR4_Selections.png" alt="River Tracing GBR4 Selections" width="280">